In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
rating_data = pd.read_csv("animelist.csv")
anima_data = pd.read_csv("anime.csv")

# To save my Pc time I made a new data where i just fetch anime_id(MAL_ID) and Name so that i can use merge() function on it.
anima_data = anima_data.rename(columns={"MAL_ID": "anime_id"})
anima_contact_data = anima_data[["anime_id", "Name"]]

In [12]:
len(rating_data)

109224747

In [13]:
rating_data = rating_data.merge(anima_contact_data, left_on = 'anime_id', right_on = 'anime_id', how = 'left')
rating_data = rating_data[["user_id", "Name", "anime_id","rating", "watching_status", "watched_episodes"]]

In [14]:
count = rating_data['user_id'].value_counts()
count1 = rating_data['anime_id'].value_counts()
rating_data = rating_data[rating_data['user_id'].isin(count[count >= 500].index)].copy()
rating_data = rating_data[rating_data['anime_id'].isin(count1[count1 >= 200].index)].copy()

In [15]:
combine_movie_rating = rating_data.dropna(axis = 0, subset = ['Name'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['Name'])['rating'].
     count().
     reset_index()
     [['Name', 'rating']]
    )

In [16]:
rating_data = combine_movie_rating.merge(movie_ratingCount, left_on = 'Name', right_on = 'Name', how = 'left')
rating_data = rating_data.drop(columns = "rating_x")
rating_data = rating_data.rename(columns={"rating_y": "rating"})

In [17]:
# Encoding categorical data
user_ids = rating_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_data["user"] = rating_data["user_id"].map(user2user_encoded)
n_users = len(user2user_encoded)

anime_ids = rating_data["anime_id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_data["anime"] = rating_data["anime_id"].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min total rating: {}, Max total rating: {}".format(min(rating_data['rating']), max(rating_data['rating'])))

Num of users: 66698, Num of animes: 11812
Min total rating: 138, Max total rating: 60305


In [18]:
g = rating_data.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rating_data.join(top_users, rsuffix='_r', how='inner', on='user_id')

g = rating_data.groupby('anime_id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='anime_id')

pivot = pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc=np.sum)

In [19]:
pivot.fillna(0, inplace=True)

In [20]:
piviot_table = rating_data.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)

In [21]:
from scipy.sparse import csr_matrix
piviot_table_matrix = csr_matrix(piviot_table.values)

In [22]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [31]:

def predict():
    random_anime = np.random.choice(piviot_table.shape[0]) # This will choose a random anime name and our model will predict on it.
    random_anime = 7
    print(random_anime)

    query = piviot_table.iloc[random_anime, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)
    
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))



In [32]:
predict()

7
Recommendations for "Star"t:

1: Shout Our Evidence!, with distance of 0.1623989551091437:
2: Agitation!, with distance of 0.17898515864964504:
3: Issei Kiyuukyoku: Divertimento, with distance of 0.1794972880862502:
4: Ever Green Never End, with distance of 0.2156511917752274:
5: Sayonara Freeway, with distance of 0.21981050239450917:


In [38]:
anima_data.anime_id[anima_data.Name == '"Star"t']

16500    41090
Name: anime_id, dtype: int64

## МОЕ предсказание! 

In [42]:
import pandas as pd
import numpy as np
import sqlite3
import re
from collections import Counter

conn = sqlite3.connect('anime.db')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_row', 200)

df = pd.read_csv('anime.csv', index_col='MAL_ID')
my_anime = pd.read_sql('SELECT * FROM anime', conn)

In [43]:
type_dict = {
    'TV': 'ТВ',
    'Movie': 'Полнометражный фильм',
    'OVA': 'OVA',
    'Special': 'ТВ-спэшл',
    'ONA': 'ONA',
    'Music': 0,
    'Unknown': 0
}
df.Type = df.Type.apply(lambda x: type_dict[x])
df2 = df[(df.Type != 0) & (df.Aired != 'Unknown') & (df.Episodes != 'Unknown')]
df2.Aired = df2.Aired.apply(lambda x: x if len(x) < 15 else x[:15])

C:\Users\PC\AppData\Local\Temp\ipykernel_19672\627347272.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.Aired = df2.Aired.apply(lambda x: x if len(x) < 15 else x[:15])


In [44]:
to_anime_name_list = [el.lower().strip() for el in df.Name.values]
my_anime_name_list = my_anime.name_en.values
to_set = set(to_anime_name_list)
my_set = set(my_anime_name_list)
name_name = my_set & to_set

In [45]:
to_list = [
    (
        name.lower(),
        int(re.findall(r'\d{4,}', date)[0]) if re.findall(r'\d{4,}', date) else 0,
        Type,
        int(Episodes)
        )\
            for name, date, Type, Episodes in zip(
                df2.Name.values,
                df2.Aired.values,
                df2.Type.values,
                df2.Episodes.values
                )
                if name not in name_name
]

In [46]:
my_list = [
    (name, int(date), Type, int(episode)) for name, date, Type, episode in zip(
                my_anime.name_en.values,
                my_anime.release.values,
                my_anime.type.values,
                my_anime.episodes_all.values
                )
                if name not in name_name
]   

In [47]:
df3 = pd.DataFrame(to_list)
list_dict = {}

for myName, myYear, myType, myEp in my_list:
    list_ = []
    name_list = list(df3[(df3[1] == myYear) & (df3[2] == myType) & (df3[3] == myEp)][0].values)
    if name_list:
        for word in myName.split():
            for toName in name_list:
                if word in toName:
                    list_.append(toName)
        if list_:
            n, c = Counter(list_).most_common(1)[0]
            list_dict.update({myName: n})

In [48]:
list_duble = [el[0] for el in Counter(list_dict.values()).most_common(29)]
not_duble = [(k, v) for k, v in list_dict.items() if v not in list_duble]
duble = [(k, v) for k, v in list_dict.items() if v in list_duble]

In [49]:
l = [
    ('bokutachi wa benkyou ga dekinai 2nd season', 'bokutachi wa benkyou ga dekinai'),
    ('gintama (2017)', 'gintama.'),		
    ('high school of the dead', 'highschool of the dead'),		
    ('ikoku meiro no croisee the animation', 'ikoku meiro no croisée the animation'),		
    ('kaitou joker 3nd season', 'kaitou joker 3rd season'),	
    ('kuroko no basuke 2', 'kuroko no basket 2nd season'),		
    ('uchuu senkan tiramisu', 'uchuu senkan tiramisù'),		
    ('yuru camp 2nd season', 'yuru camp△ season 2')		
]
not_duble += l

In [50]:
to_dict_encoder = {name.lower().strip() : id for name, id in zip(df.Name.values, df.index)}
my_dict_encoder = {name : id for name, id in zip(my_anime.name_en.values, my_anime.anime_id.values)}

In [51]:
result_dict = {}
for name in name_name:
    result_dict.update({my_dict_encoder[name] : to_dict_encoder[name]})

for my, to in not_duble:
    toId = to_dict_encoder[to]
    if toId not in result_dict.values():
        result_dict.update({my_dict_encoder[my] : to_dict_encoder[to]})

In [52]:
inverted_result_dict = {v: k for k, v in result_dict.items()}

In [88]:
def predict(n):
    random_anime = np.random.choice(piviot_table.shape[0]) # This will choose a random anime name and our model will predict on it.
    random_anime = n
    print(random_anime)

    query = piviot_table.iloc[random_anime, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)
    
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))



In [129]:
predict(1)

1
Recommendations for "Bungaku Shoujo" Kyou no Oyatsu: Hatsukoi:

1: "Bungaku Shoujo" Memoire, with distance of 0.2728255655632089:
2: "Bungaku Shoujo" Movie, with distance of 0.41644578327285564:
3: Black★Rock Shooter (OVA), with distance of 0.6382134953827137:
4: Kowarekake no Orgel, with distance of 0.6530747144992612:
5: Hanbun no Tsuki ga Noboru Sora, with distance of 0.6557215440553388:


In [85]:
indexes = inverted_result_dict.keys()
true_indexes = {}
for i, name in enumerate(piviot_table.index.to_list()):
    j = anima_data.anime_id[anima_data.Name == name].values[0]
    if j in indexes:
        true_indexes.update({i:inverted_result_dict[j]})
len(true_indexes)

2262

In [98]:
3 in true_indexes.keys()

False

In [154]:
indexes = list(true_indexes.keys())
result = []

for i in indexes:
    count = 0
    id  = true_indexes[i]
    list_ = []
    query = piviot_table.iloc[i, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=200)

    for j in range(1, len(distance.flatten())):
        index = suggestions.flatten()[j]

        if index in indexes:     
            count += 1       
            list_.append((int(id), int(true_indexes[index]), count))  
                     
            if count == 20:
                break
    
    result += list_

In [157]:

len(result), len(indexes) * 20

(44656, 45240)

In [156]:
conn = sqlite3.connect('anime.db')
cursor = conn.cursor()
cursor.executemany('INSERT INTO recommendation values(?, ?, ?)', result)
conn.commit()
cursor.close()
conn.close()

In [133]:
result

[(420, 438, 1, 0.2728255655632089),
 (420, 86, 2, 0.6382134953827137),
 (420, 1573, 3, 0.6557215440553388),
 (420, 1541, 4, 0.6589879627453901),
 (420, 373, 5, 0.6637004520350145),
 (420, 1526, 6, 0.6652111329149939),
 (420, 436, 7, 0.6688716902178211),
 (420, 1698, 8, 0.6708644157767671),
 (420, 314, 9, 0.6711037568919007),
 (420, 1552, 10, 0.6714325421509069)]

In [70]:
true_indexes

[8319    20707
 Name: anime_id, dtype: int64,
 5112    7669
 Name: anime_id, dtype: int64,
 5393    8481
 Name: anime_id, dtype: int64,
 4584    6408
 Name: anime_id, dtype: int64,
 4448    6076
 Name: anime_id, dtype: int64,
 11863    33363
 Name: anime_id, dtype: int64,
 12499    34595
 Name: anime_id, dtype: int64,
 16500    41090
 Name: anime_id, dtype: int64,
 16224    40554
 Name: anime_id, dtype: int64,
 16283    40658
 Name: anime_id, dtype: int64,
 2674    2928
 Name: anime_id, dtype: int64,
 2919    3269
 Name: anime_id, dtype: int64,
 3657    4469
 Name: anime_id, dtype: int64,
 425    454
 Name: anime_id, dtype: int64,
 1039    1143
 Name: anime_id, dtype: int64,
 275    299
 Name: anime_id, dtype: int64,
 5717    9332
 Name: anime_id, dtype: int64,
 6193    10390
 Name: anime_id, dtype: int64,
 788    873
 Name: anime_id, dtype: int64,
 29    48
 Name: anime_id, dtype: int64,
 274    298
 Name: anime_id, dtype: int64,
 1816    1994
 Name: anime_id, dtype: int64,
 6529    1

In [55]:
len(piviot_table)

11810